In [1]:
#uses Registered#Nto1Adder_NIR
#registerings implemented based on the assumption that delay of all Registered#Nto1Adder_NIR is 1 clk

In [2]:
import sys
import math
import io

In [3]:
#N>=2
N = input('Enter N: ')

Enter N: 14


In [4]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10): ')

Enter IN_WIDTH default value (empty for 10): 


In [5]:
#AdderSize>=2
AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')

Enter maximum size allowed for adders(empty for 2): 4


In [6]:
N = int(N)
if N<2:
    N=2
if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if IN_WIDTH<1:
        IN_WIDTH=1
if not AdderSize:
    AdderSize=2
else:
    AdderSize = int(AdderSize)
    if AdderSize<2:
        AdderSize=2

In [7]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of=open("./AdderTree_{}_A{}.v".format(N, AdderSize), 'w+')

In [8]:
print("`timescale 1ns / 1ps\n", file=of)
print("module AdderTree_{}_A{}".format(N, AdderSize), file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("output outReady,", file=of)
lgn=math.ceil(math.log(N,2))
al=-1+lgn
if al==0:
    print("output signed [IN_WIDTH:0] out,", file=of)
else:
    print("output signed [IN_WIDTH+{}:0] out,".format(al), file=of)
print("output earlyOutReady", file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module AdderTree_14_4
#(
parameter IN_WIDTH = 10
)(
input clk, reset, enable,
input inReady,
input signed [IN_WIDTH-1:0] A0, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, 
output outReady,
output signed [IN_WIDTH+3:0] out,
output earlyOutReady
);



In [9]:
AIBuf=io.StringIO()

level=0

nextElememtNo=N #remained
sizeList=[]
for i in range(N):
    sizeList.append(1)

RootList=[]
for i in range(N):
    RootList.append(i)
AdditionalRegList=[]
for i in range(N):
    AdditionalRegList.append(0)

i=0
new = N

while 1:
    #print("StartNextElememtNo = {}".format(nextElememtNo))
    if nextElememtNo==1:
        break
    if nextElememtNo<=AdderSize:
        lastLevel=True
    else:
        lastLevel=False        
    currentElememtNo = nextElememtNo
    nextElememtNo=0
    ORimp=False
    while currentElememtNo>0:
        if currentElememtNo >= AdderSize:
            CurrentAdderSize = AdderSize
            currentElememtNo -= AdderSize
        else:
            if not lastLevel:
                CurrentAdderSize = 1
                currentElememtNo -= 1
            else:
                CurrentAdderSize = currentElememtNo
                currentElememtNo = 0
        nextElememtNo=nextElememtNo+1
        #print("nextElememtNo = {}".format(nextElememtNo))
        outSize=0
        for k in range (i, i+CurrentAdderSize):
            outSize+=sizeList[k]
        sizeList.append(outSize)
        if CurrentAdderSize==1: #just reg
            root=RootList[i]
            if root != -1:
                RootList.append(i)
                #print(root)
                while root>=N:
                    root=RootList[root]
                    #print(root)
                AdditionalRegList[root]+=1
            else:
                RootList.append(-1)
            i+=1
        else: #Generate a AdderSize Adder (maybe less size for last stage)
            RootList.append(-1)
            if not lastLevel:
                if outSize==2: #outSize>=2
                    AIBuf.write("wire signed [IN_WIDTH:0] M{};\n".format(new))
                else:
                    AIBuf.write("wire signed [IN_WIDTH+{}:0] M{};\n".format(math.ceil(math.log(outSize,2))-1, new))
            inSizeL = math.ceil(math.log(sizeList[i],2))
            #print("inSize = {}".format(inSize))
            rE = i+CurrentAdderSize
            eSizeList=[]
            finalRootList=[]
            for k in range (i, rE):
                curSizeL=math.ceil(math.log(sizeList[k],2))
                eSizeL=inSizeL-curSizeL
                eSizeList.append(eSizeL)
                #print("esize = {}".format(eSizeL))
                root=RootList[k]
                if root != -1:
                    while root>=N:
                        root=RootList[root]
                else:
                    root=k
                finalRootList.append(root)
                if eSizeL!=0:
                    AIBuf.write("wire signed [2*IN_WIDTH")
                    if inSizeL!=1:
                         AIBuf.write("+{}".format(inSizeL-1, k))
                    if(root<N):
                        AIBuf.write(":0] A{}e = ".format(root))
                    else:
                        AIBuf.write(":0] M{}e = ".format(root))
                    AIBuf.write("{{")
                    AIBuf.write("{}".format(eSizeL))
                    AIBuf.write("{")
                    if(root<N):
                        AIBuf.write("A{}".format(root))
                    else:
                        AIBuf.write("M{}".format(root))
                    AIBuf.write("[2*IN_WIDTH".format(k))
                    if curSizeL==0:
                        AIBuf.write("-1")
                    elif curSizeL>1:
                        AIBuf.write("+{}".format(curSizeL-1))
                    AIBuf.write("]}},")
                    if(root<N):
                        AIBuf.write("A{}".format(root))
                    else:
                        AIBuf.write("M{}".format(root))
                    AIBuf.write("};\n")
            AIBuf.write("Registered{}to1Adder_NIR #(.IN_WIDTH(IN_WIDTH".format(CurrentAdderSize))
            curSizeL=math.ceil(math.log(sizeList[i],2))
            if inSizeL>0:
                AIBuf.write("+{}".format(inSizeL))
            AIBuf.write("))\nAdd{}(\nclk, reset, enable, \n".format(new))
            if level==0:
                AIBuf.write("inReady,\n")
            else:
                AIBuf.write("OR[{}],\n".format(level-1))
            for k in range (i, rE):
                if(finalRootList[k-i]<N):
                    AIBuf.write("A{}".format(finalRootList[k-i]))
                else:
                    AIBuf.write("M{}".format(finalRootList[k-i]))
                if(eSizeList[k-i]!=0):
                    AIBuf.write("e");
                AIBuf.write(", ")
            AIBuf.write("\n")
            if lastLevel:
                AIBuf.write("outReady,\n")
            else:
                if ORimp==False:
                    AIBuf.write("OR[{}],\n".format(level))
                    ORimp=True
                else:
                    AIBuf.write("ORa{}, //not used\n".format(new))
            if lastLevel:
                AIBuf.write("out,\n".format(new))
            else:
                AIBuf.write("M{},\n".format(new))
            if lastLevel:
                AIBuf.write("earlyOutReady);\n\n")        
            else :
                AIBuf.write("aeor{}); //not used\n\n".format(new))
            i+=CurrentAdderSize
        new+=1
    level+=1

In [10]:
print(RootList)
print(AdditionalRegList)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -1, -1, -1, 12, 13, -1, 18, -1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2]


In [11]:
if level>1:
    print("wire [0:{}] OR;\n".format(level-2), file=of)
    
print(AIBuf.getvalue(), end='',file=of)

print("endmodule", file=of)

wire [0:1] OR;

wire signed [IN_WIDTH+1:0] M14;
Registered4to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))
Add14(
clk, reset, enable, 
inReady,
A0, A1, A2, A3, 
OR[0],
M14,
aeor14); //not used

wire signed [IN_WIDTH+1:0] M15;
Registered4to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))
Add15(
clk, reset, enable, 
inReady,
A4, A5, A6, A7, 
ORa15, //not used
M15,
aeor15); //not used

wire signed [IN_WIDTH+1:0] M16;
Registered4to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))
Add16(
clk, reset, enable, 
inReady,
A8, A9, A10, A11, 
ORa16, //not used
M16,
aeor16); //not used

wire signed [IN_WIDTH+3:0] M19;
wire signed [2*IN_WIDTH+1:0] A12e = {{2{A12[2*IN_WIDTH-1]}},A12};
Registered4to1Adder_NIR #(.IN_WIDTH(IN_WIDTH+2))
Add19(
clk, reset, enable, 
OR[0],
M14, M15, M16, A12e, 
OR[1],
M19,
aeor19); //not used

wire signed [2*IN_WIDTH+3:0] A13e = {{4{A13[2*IN_WIDTH-1]}},A13};
Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH+4))
Add21(
clk, reset, enable, 
OR[1],
M19, A13e, 
outReady,
out,
earlyOutReady);

endmodule


In [12]:
if __Print_To_File>0:
    of.close()